In [29]:
import numpy as np
import cupy as cp
import time

# CUDA kernel for computing the sum
sum_kernel = cp.RawKernel(r'''
extern "C" __global__
void sum_kernel(const double* x, double* y, int n) {
    extern __shared__ double sdata[];
    
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (i < n) ? x[i] : 0.0;
    __syncthreads();

    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    if (tid == 0) {
        y[blockIdx.x] = sdata[0];
    }
}
''', 'sum_kernel')

def gpu_sum_kernel(arr):
    arr_gpu = cp.asarray(arr)
    n = arr_gpu.size

    threads_per_block = 1024
    blocks_per_grid = (n + threads_per_block - 1) // threads_per_block

    block_sums = cp.zeros(blocks_per_grid, dtype=cp.float64)

    shared_mem_size = threads_per_block * arr_gpu.itemsize

    sum_kernel((blocks_per_grid,), (threads_per_block,), (arr_gpu, block_sums, n), shared_mem=shared_mem_size)

    total = cp.sum(block_sums)
    return total

def cpu_sum(arr):
    total = 0.0
    for val in arr:
        total += val
    return total

if __name__ == "__main__":
    size = 4096000
    data = np.random.random(size)

    start_cpu = time.time()
    cpu_total = cpu_sum(data)
    end_cpu = time.time()

    cpu_duration = end_cpu - start_cpu
    print(f"CPU Result: {cpu_total}, Execution Time: {cpu_duration:.6f} sec")

    start_gpu = time.time()
    gpu_total = gpu_sum_kernel(data).get()
    end_gpu = time.time()

    gpu_duration = end_gpu - start_gpu
    print(f"GPU Result (kernel): {gpu_total}, Execution Time: {gpu_duration:.6f} sec")

CPU Result: 1023916.8375448914, Execution Time: 0.210000 sec
GPU Result (kernel): 1023916.8375448463, Execution Time: 0.014000 sec
